In [72]:
import pandas as pd
import numpy as np

from context import community_module
from community_module.community_detection.similarityCommunityDetection import SimilarityCommunityDetection
from community_module.community_detection.explainedCommunitiesDetection import ExplainedCommunitiesDetection
from community_module.visualization.gephiVisualization import GephiVisualization

from sklearn.metrics.pairwise import cosine_similarity
from community_module.similarity.taxonomySimilarity import TaxonomySimilarity

In [73]:
interactions_df = pd.read_csv('../../data/IMMA/citizenInteractions.csv')
interactions_df.head()

,userResponseID,artworkID,numInteractions
0,81PfE8l9tbFhtvxOE6X5VOh2UpBiYgCO,60c0927e0ee8234e932e7a1d,1
1,BZXf8rM1IAoDUtW01ShxYw8MxjHtGF5S,60c08c580ee8234e932e7a18,1
2,BZXf8rM1IAoDUtW01ShxYw8MxjHtGF5S,60c0908cb5b26479a17d1b33,1
3,BZXf8rM1IAoDUtW01ShxYw8MxjHtGF5S,60c09225b5b26479a17d1b3a,1
4,BZXf8rM1IAoDUtW01ShxYw8MxjHtGF5S,60c0927e0ee8234e932e7a1d,1


In [74]:
artworks = pd.read_excel('../../data/IMMA/artworks_IMMA_enhanced.xlsx')
artworks

,_id,name,artist,date,description,notes,URL,image,audio,_datasetid,decade,type,medium,artist_country
0,60c095321262e65eec197529,Back of Snowman,Gary Hume,2003,Gary Hume was one of a generation of artists w...,"IMMA Collection: Purchase, 2005",https://imma.ie/collection/back-of-snowman/,08GaryHume.jpg,08-BackSnowman-Joan.mp3,a6ad7a45-3d69-44c2-8f57-e0830e748f0d,2000,Sculpture,Bronze,England
1,60c094dcb78b241f4625bbb9,SENTINEL VIII,Catherine Lee,1999,"Born in Texas in 1950, Lee’s works are a hybri...","IMMA Collection: Donation, Sean Scully, 2005",https://imma.ie/collection/sentinel-viii/,07CatherineLee.jpg,07-Sentinel-Joan.mp3,a6ad7a45-3d69-44c2-8f57-e0830e748f0d,1990,Sculpture,Bronze,USA
2,60c09476b84c4b312635d34c,Barrel,Fergus Martin,2020,"This artwork is by Fergus Martin, an Irish art...",Commissioned by the Office for Public Works fo...,NaN,06FergusMartin.jpg,06-Barrel-Joan.mp3,a6ad7a45-3d69-44c2-8f57-e0830e748f0d,2020,Sculpture,Stainless Steel,Ireland
3,60c09437b5b26479a17d1b3c,217 5° Arc x 12,Bernar Venet,2008,"Born in 1941 in the south of France, Venet est...","IMMA Collection: Loan, collection of the artis...",https://imma.ie/collection/217-5-arc-x-12/,05BernarVenet.jpg,05-Arc-Joan.mp3,a6ad7a45-3d69-44c2-8f57-e0830e748f0d,2000,Sculpture,Cor-ten Steel,France
4,60c0927e0ee8234e932e7a1d,The Drummer,Barry Flanagan,1996,"Born in Wales, Barry Flanagan studied at Birmi...","IMMA Collection: Donated by the artist, 2001",https://imma.ie/collection/the-drummer/,04BarryFlanagan.jpg,04-Drummer-Joan.mp3,a6ad7a45-3d69-44c2-8f57-e0830e748f0d,1990,Sculpture,Bronze,Wales
5,60c09225b5b26479a17d1b3a,8 Limestones cut to a specific size from rough...,Ulrich Rückriem,1988,Ulrich Rückreim is German and he lives in Colo...,"IMMA Collection: Purchase, 2001",https://imma.ie/collection/8-limestones-cut-to...,03UrichRuckreim.jpg,03-8Limestones-Joan.mp3,a6ad7a45-3d69-44c2-8f57-e0830e748f0d,1980,Sculpture,Limestone,Germany
6,60c0908cb5b26479a17d1b33,Untitled / Corrections D,Iran do Espírito Santo,2008,<p>Brazilian artist Iran do Espírito Santo’s w...,"IMMA Collection: Purchase, 2007",https://imma.ie/collection/untitled-correction...,02IrandoEspiritoSanto.jpg,02-UntitledCorrectionsD-Joan.mp3,a6ad7a45-3d69-44c2-8f57-e0830e748f0d,2000,Sculpture,Granite,Brazil
7,60c08c580ee8234e932e7a18,Recurring Line: North/ South,Donald Urquhart,2007,<p>Scottish artist Donald Urquhart works in a ...,"IMMA Collection: Purchase, 2005",https://imma.ie/collection/recurring-line-nort...,01DonaldUrquhart.jpg,01-RecurringLine-Joan.mp3,a6ad7a45-3d69-44c2-8f57-e0830e748f0d,2000,Painting,Snowdrop plants,Scotland


In [75]:
countryArtworks = interactions_df.apply(lambda row: artworks[artworks['_id'] == row['artworkID']].artist_country.values[0], axis=1)
countryArtworks

0        Wales
1     Scotland
2       Brazil
3      Germany
4        Wales
5       France
6      Ireland
7          USA
8      England
9       Brazil
10     Germany
11       Wales
12         USA
13       Wales
14     Ireland
15     England
16    Scotland
17      Brazil
18     Germany
19       Wales
20     Ireland
21     England
22    Scotland
23    Scotland
24     Germany
25       Wales
26      France
27     Ireland
28         USA
29     England
30      France
31         USA
32      France
33      Brazil
dtype: object

In [76]:
names_arts = interactions_df.apply(lambda row: artworks[artworks['_id'] == row['artworkID']].name.values[0], axis=1)
interactions_df['artworkID'] = names_arts
interactions_df['country'] = countryArtworks
interactions_df

,userResponseID,artworkID,numInteractions,country
0,81PfE8l9tbFhtvxOE6X5VOh2UpBiYgCO,The Drummer,1,Wales
1,BZXf8rM1IAoDUtW01ShxYw8MxjHtGF5S,Recurring Line: North/ South,1,Scotland
2,BZXf8rM1IAoDUtW01ShxYw8MxjHtGF5S,Untitled / Corrections D,1,Brazil
3,BZXf8rM1IAoDUtW01ShxYw8MxjHtGF5S,8 Limestones cut to a specific size from rough...,1,Germany
4,BZXf8rM1IAoDUtW01ShxYw8MxjHtGF5S,The Drummer,1,Wales
5,BZXf8rM1IAoDUtW01ShxYw8MxjHtGF5S,217 5° Arc x 12,1,France
6,BZXf8rM1IAoDUtW01ShxYw8MxjHtGF5S,Barrel,1,Ireland
7,BZXf8rM1IAoDUtW01ShxYw8MxjHtGF5S,SENTINEL VIII,1,USA
8,BZXf8rM1IAoDUtW01ShxYw8MxjHtGF5S,Back of Snowman,1,England
9,KL32ozS4irwGu3JtWd0x5p8AV9kXlS9I,Untitled / Corrections D,3,Brazil


In [77]:
users_countryArtwork = pd.pivot_table(interactions_df, values='numInteractions', index='userResponseID', columns='country', fill_value=0, aggfunc=np.sum)
users_countryArtwork

country,Brazil,England,France,Germany,Ireland,Scotland,USA,Wales
userResponseID,,,,,,,,
81PfE8l9tbFhtvxOE6X5VOh2UpBiYgCO,0,0,0,0,0,0,0,1
BZXf8rM1IAoDUtW01ShxYw8MxjHtGF5S,1,1,1,1,1,1,1,1
KL32ozS4irwGu3JtWd0x5p8AV9kXlS9I,3,0,0,3,0,0,1,3
P4M8Uto1UwzTd3OKOns3LZ7hIxBheAg1,0,0,0,0,1,0,0,1
SUw10XRrB4A03Ral1MKas4sLgIW9P3lh,0,1,0,0,0,0,0,0
ScOlinhmp6Rxo9aqywrKMR8J0S90qNQA,2,2,0,2,1,2,0,2
ZqNE3fNu8JXVIadgc5epXYofaOtgEE7F,0,0,0,0,0,1,0,0
aQIxJWU4S2YAOGA93ItHtmuEeNbhudnY,0,1,1,1,1,1,1,1
iCY4NVfyljZxazkltmZh5YiAnY2b3Rm4,0,0,6,0,0,0,3,0


In [78]:
community_detection = ExplainedCommunitiesDetection(users_countryArtwork, SimilarityCommunityDetection, TaxonomySimilarity)

In [79]:
n_communities, users_communities = community_detection.search_all_communities(percentage=0.78)

In [80]:
users_countryArtwork['community'] = users_communities.values()
users_countryArtwork.reset_index(inplace=True)
users_countryArtwork

country,userResponseID,Brazil,England,France,Germany,Ireland,Scotland,USA,Wales,community
0,81PfE8l9tbFhtvxOE6X5VOh2UpBiYgCO,0,0,0,0,0,0,0,1,5
1,BZXf8rM1IAoDUtW01ShxYw8MxjHtGF5S,1,1,1,1,1,1,1,1,2
2,KL32ozS4irwGu3JtWd0x5p8AV9kXlS9I,3,0,0,3,0,0,1,3,6
3,P4M8Uto1UwzTd3OKOns3LZ7hIxBheAg1,0,0,0,0,1,0,0,1,7
4,SUw10XRrB4A03Ral1MKas4sLgIW9P3lh,0,1,0,0,0,0,0,0,1
5,ScOlinhmp6Rxo9aqywrKMR8J0S90qNQA,2,2,0,2,1,2,0,2,3
6,ZqNE3fNu8JXVIadgc5epXYofaOtgEE7F,0,0,0,0,0,1,0,0,4
7,aQIxJWU4S2YAOGA93ItHtmuEeNbhudnY,0,1,1,1,1,1,1,1,1
8,iCY4NVfyljZxazkltmZh5YiAnY2b3Rm4,0,0,6,0,0,0,3,0,0
9,k5SXzdwN9UPY2FXUC1vavDtk3CykTKXR,0,0,1,0,0,0,0,0,0


In [81]:
# Pintamos en Gephi
gv = GephiVisualization(workspace='imma_country_taxonomy')

# Preparamos los datos de usuarios y distancias
users = users_countryArtwork[['userResponseID','community']].values
data = users_countryArtwork.iloc[:,1:-1].values
distances = cosine_similarity(data)

gv.load_community(users, distances, users_properties=['community'])

In [82]:
users_countryArtwork[['userResponseID', 'community']]

country,userResponseID,community
0,81PfE8l9tbFhtvxOE6X5VOh2UpBiYgCO,5
1,BZXf8rM1IAoDUtW01ShxYw8MxjHtGF5S,2
2,KL32ozS4irwGu3JtWd0x5p8AV9kXlS9I,6
3,P4M8Uto1UwzTd3OKOns3LZ7hIxBheAg1,7
4,SUw10XRrB4A03Ral1MKas4sLgIW9P3lh,1
5,ScOlinhmp6Rxo9aqywrKMR8J0S90qNQA,3
6,ZqNE3fNu8JXVIadgc5epXYofaOtgEE7F,4
7,aQIxJWU4S2YAOGA93ItHtmuEeNbhudnY,1
8,iCY4NVfyljZxazkltmZh5YiAnY2b3Rm4,0
9,k5SXzdwN9UPY2FXUC1vavDtk3CykTKXR,0


In [83]:
users_without_community = []
for c in range(n_communities):
    community_data = community_detection.get_community(c, answer_binary=True)

    if len(community_data['members']) > 1:

        print('---------------------')
        print('COMMUNITY -', community_data['name'])
        print('\t- N. Members:', len(community_data['members']))
        print('\t- Properties:')

        for k in community_data['properties'].keys():
            print('\t\t-', k)
    else:
        users_without_community.extend(community_data['members'])

print('---------------------')
print('N. USERS WITHOUT COMMUNITY -', len(users_without_community))

---------------------
COMMUNITY - 0
	- N. Members: 2
	- Properties:
		- France
		- USA
---------------------
COMMUNITY - 1
	- N. Members: 2
	- Properties:
		- England
---------------------
COMMUNITY - 2
	- N. Members: 2
	- Properties:
		- Brazil
---------------------
N. USERS WITHOUT COMMUNITY - 5
